In [52]:
import numpy             as np
import matplotlib.pyplot as plt
import csv 
import pickle

import ipywidgets      as widgets
from   ipywidgets      import Button, HBox, VBox
from   IPython.display import display
from   IPython.display import clear_output
from IPython.display import display, HTML
from IPython.display import FileLink, FileLinks




filename   = 'CIFAR_10_Test_Batch-Copy1'
filename2 = 'Batch Names-Copy1'

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

file = unpickle(filename)
file2 = unpickle(filename2)


images = file[b'data'] 
labels = file[b'labels']
label_names = file2[b'label_names']


images = images.reshape(len(labels), 3, 32, 32)

def imagshow(img):
    plt.imshow(np.transpose(img, (1, 2, 0))) #flip the grid from side to normal
    plt.show()
# show the image 
    
exImages = np.random.randint(0,len(labels))


#example image 

dataset = {}
total = len(images)
image_ids = np.arange(total)
images_per_person = 200
people = int(total/images_per_person)

for I in range(people):
    ind1 = I*images_per_person
    ind2 = (I+1)*images_per_person
    dataset[I] = images[ind1:ind2,:,:,:]
    
results= {}    

I = 0
#variable used, original value is 0
answers = []
#The results of people's answers in the survey
outfile = 'ans_'+filename
#A file with all of the answers (I create this one)
datalen = len(labels)

def user_id_images(b):
    userid.close()
    submit.close()
    V = userid.value
    images = dataset[V]
    
    with out:
        imagshow(images[V])
    display(out)
    
    display(VBox([widgetsBox, buttonNext]))
    display()
    display(VBox([done]))
    
def on_buttonNext_clicked(b):
    R = userid.value
    images = dataset[R]
    global V 
    #Only changes I for individual actions
    V = V+1
    if V < images_per_person:
        with out: 
        #Do the things within "out" - which is the widget output
            row = []
        #The newlist that the answers will be appended to
            for l in range(0,len(widget_labels)): #Creating widget answers by changing the individual widget values
                row.extend([items[l].value]) #appending the widget answers 
                items[l].value = 0 #Resets the values 
            answers.append(row) #Puts in the new row answers into the overall answers
            ID = (images_per_person*R)+V
            results[ID] = row, labels[ID-1]
            clear_output() #Resets the widget numbers
            imagshow(images[V]) #Displays a new image
            if I % 5 == 0: #Saves answers
                saveAnswers(b)
    else:
        finished_survey(b)

def finished_survey(b):
    widgetsBox.close()
    buttonNext.close()
    out.close()
    done.close()
    I_am_sure.close()
    print("""
Thank you! Written below are the results from your survey. 
Please copy and paste these results and send them to Ophir at: 
ophirg543@gmail.com

""") 
    print(results)



        
def saveAnswers(b):
    image = images.reshape(10000, 3*32*32)
    #Convert the images back into normal data
    np.savez(outfile, answers=answers, images=image, labels=labels, label_names=label_names)
    
    #Save the new outfile(with new answers), new answers, new images, new labels, and new label names

            


# Image Survey

### Instructions for the survey:

There will be displayed an image in front of you. You must go through each category below the image and choose the probability that the image may be the category given. 

Give a rating on how sure you are about the image belonging to each category:

4 = You can say with 100% certainty that the photo definitely belongs to this category <br>
3 = It's probably this category, but there are details that can be interpreted as something else <br>
2 = The image might be in this category, but definitely can be in another category too <br>
1 = There's a small chance that the image is part of this category <br> 
0 = There is no way this image falls under this category and there are no details of this in the image <br>

Try to choose at least 3 categories to be non-zero. 

You will receive around 200 photos to look at and rate how much they look like each object. This should take around 2 hours. Once you are done rating the photos, the survey will naturally finish. If you do not do not have time to complete rating the photos, press the button "done!". After you press the button, press the button "I am sure I am done". DO NOT press these buttons if you are not finished, as you will have to restart your work. Let Ophir know if you have any questions. 


In [53]:
widget_labels = ['airplane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
confidence    = [4, 3, 2, 1, 0]



items  = [widgets.RadioButtons(options=confidence, 
                layout=widgets.Layout(width='50px', height='150px'), value = 0) for l in widget_labels] 
#Actually displays the widgets and numbers
buttonNext = Button(description='Next')
widgetsBox = HBox([VBox([widgets.Label(widget_labels[l]), items[l]]) for l in range(0,len(widget_labels))])
out        = widgets.Output()

userid = widgets.IntText(
    value=0,
    description='USER ID#:',
    disabled=False)

done = widgets.Button(description= 'Done!')



submit = widgets.Button(description="Submit")

display(VBox([userid,submit]))

V = userid.value

submit.on_click(user_id_images)
buttonNext.on_click(on_buttonNext_clicked)
done.on_click(finished_survey)